In [24]:
from pathlib import Path

import numpy as np
import pandas as pd
from numba import njit, prange

from matplotlib import pyplot as plt

In [25]:
fig, axes = plt.subplots(nrows=1, ncols=10)

fig.set_figwidth(10)
fig.set_dpi(150)

nums_mtrx = np.loadtxt(Path('.', 'basic', 'numbers.txt'))

for n, ax in zip(nums_mtrx, axes.ravel()):
    ax.imshow(n.reshape(5, 3), cmap='gray')

plt.tight_layout()
plt.show()

In [26]:
cells_columns = [f'c{n}' for n in range(15)]
classes = ['zero', 'one', 'two', 'three', 'four',
           'five', 'six', 'seven', 'eight', 'nine']

df = pd.DataFrame(columns=cells_columns+['class'])

for i in range(10):
    new_row = dict(zip(cells_columns, nums_mtrx[i]))
    new_row['class'] = classes[i]

    df.loc[df.index.size] = new_row

    for j in range(9):
        noised_num = nums_mtrx[i] + np.random.uniform(0, 0.5, 15)

        new_row = dict(zip(cells_columns, noised_num))
        new_row['class'] = classes[i]

        df.loc[df.index.size] = new_row

for name in df['class'].unique():
    df[f'{name}_label'] = df['class'].map(lambda x: 1 if x == name else 0)

In [ ]:
test_frac = 0.7

p = np.arange(10)

test_size = int(p.size*test_frac)
train_size = int(p.size*(1-test_frac))

idx_test = np.ravel([(p[0: test_size] + i*10) for i in range(10)])
idx_train = np.ravel([(p[test_size: p.size] + i*10) for i in range(10)])

features_columns = cells_columns
label_columns = [f"{name}_label" for name in classes]

xTest = np.array(df.iloc[idx_test][features_columns])
yTest = np.array(df.iloc[idx_test][label_columns])

xTrain = np.array(df.iloc[idx_train][features_columns])
yTrain = np.array(df.iloc[idx_train][label_columns])

In [27]:
@njit(fastmath=True)
def F(x: np.ndarray) -> np.ndarray:
    # return x
    # return np.clip(x,-1, 1)
    return np.tanh(x)


@njit(fastmath=True)
def dF(x: np.ndarray) -> np.ndarray:
    # return np.ones(x.shape)
    # return np.array([0 if xi <= -1 or xi >= 1 else 1 for xi in x])
    return 1-np.square(np.tanh(x))


@njit(fastmath=True)
def relu(x: np.ndarray) -> np.ndarray:
    return np.maximum(np.zeros(x.shape), x)


@njit(fastmath=True)
def dRelu(x: np.ndarray) -> np.ndarray:
    return 1 * (x > 0)


@njit(fastmath=True)
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1/(1+np.exp(-x))


@njit(fastmath=True)
def dSigmoid(x: np.ndarray) -> np.ndarray:
    y = 1/(1+np.exp(-x))
    return y*(1-y)


@njit(fastmath=True)
def softmax(x: np.ndarray) -> np.ndarray:
    y = np.exp(x)
    return y/np.sum(y)

In [28]:
@njit(fastmath=True)
def grads(xBatch: np.ndarray, yBatch: np.ndarray, Wh: np.ndarray, w: np.ndarray, Bh: np.ndarray, Bx: np.ndarray, b: np.ndarray) -> tuple[np.ndarray]:
    dWh = np.zeros(Wh.shape)
    dw = np.zeros(w.shape)

    dBh = np.zeros(Bh.shape)
    dBx = np.zeros(Bx.shape)
    db = np.zeros(b.shape)

    for i in prange(xBatch.shape[0]):
        h = xBatch[i] @ Wh + Bh
        z = sigmoid(h @ Wh.T + Bx)
        y = softmax(z @ w + b)

        dLdz = (y-yBatch[i]) @ w.T

        dw += np.outer(z, y-yBatch[i])
        db += y-yBatch[i]

        dWh += np.outer(xBatch[i], dLdz * z*(1-z) @ Wh)
        dBx += dLdz * z*(1-z)
        dBh += dLdz * z*(1-z) @ Wh

    return (dWh, dw, dBh, dBx, db)


class RBMPerceptron:
    def __init__(self, nH: int, nIn: int, nOut: int) -> None:
        self.nH = nH
        self.nIn = nIn
        self.nOut = nOut

        self.Wh: np.ndarray = np.random.uniform(-1, 1, (nIn, nH))
        self.w: np.ndarray = np.random.uniform(-1, 1, (nIn, nOut))

        self.Bh: np.ndarray = np.zeros(nH)
        self.Bx: np.ndarray = np.zeros(nIn)
        self.b: np.ndarray = np.zeros(nOut)

    def predict(self, x: np.ndarray) -> np.ndarray:
        h = x @ self.Wh + self.Bh
        z = h @ self.Wh.T + self.Bx
        return softmax(sigmoid(z) @ self.w + self.b)

    def train(self, xTrain: np.ndarray, yTrain: np.ndarray, lr, batch_size, max_epoch) -> None:
        n = xTrain.shape[0]

        for epoch in range(max_epoch):
            idxs = np.random.choice(a=np.arange(
                n), size=batch_size, replace=False)

            for batch_idx in range(n//batch_size):
                ibegin, iend = batch_idx * \
                    batch_size, min((batch_idx+1)*batch_size, n-1)
                batch_idxs = idxs[ibegin:iend]

                dWh, dw, dBh, dBx, db = grads(
                    xTrain[batch_idxs], yTrain[batch_idxs], self.Wh, self.w, self.Bh, self.Bx, self.b)

                self.Wh -= lr*dWh
                self.w -= lr*dw

                self.Bh -= lr*dBh
                self.Bx -= lr*dBx
                self.b -= lr*db

    def loss(self, x: np.ndarray, y: np.ndarray) -> float:
        yPred = np.array([self.predict(xi) for xi in x])
        h = -np.sum(y*np.log(yPred), axis=1)
        return 1/x.shape[0] * np.sum(h)

In [45]:
nH = 16
nIn = 4
nOut = 3

lr = 1e-2
batch_size = 8
max_epoch = 5000

model = RBMPerceptron(nH, nIn, nOut)

print('untrained loss: {0:.6f}'.format(model.loss(xTest, yTest)))

model.train(xTrain, yTrain, lr, batch_size, max_epoch)

print('trained loss: {0:.6f}'.format(model.loss(xTest, yTest)))

TP_count = 0
for x, y in zip(xTest, yTest):
    yPred = model.predict(x)
    TP_count += 1 if np.argmax(y) == np.argmax(yPred) else 0

accuracy = TP_count / xTest.shape[0]
print(f"test set accuracy: {round(accuracy*100, 2)}%")

untrained loss: 1.190275
trained loss: 0.259933
test set accuracy: 94.17%


In [ ]:
x_noised = nums_mtrx + np.random.uniform(0, 0.5, 15)

yPred = list([model.predict(xi) for xi in x_noised])

fig, axes = plt.subplots(nrows=1, ncols=10)

fig.set_dpi(100)
fig.set_figwidth(15)
fig.set_figheight(5)

for i, ax in enumerate(axes.ravel()):
    ax.imshow(x_noised[i].reshape((5, 3)),  cmap='gray')
    j = np.argmax(yPred[i])
    ax.set_title(f"{j} : {(yPred[i][j]*100).round()}%")

plt.autoscale()
plt.tight_layout()
plt.show()